In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
vanilla = pd.read_csv("./data/vanilla.csv")
vanilla["algorithm"] = "vanilla"
parfor = pd.read_csv("./data/parfor.csv")
parfor["algorithm"] = "parfor"
tiled = pd.read_csv("./data/tiled.csv")
tiled["algorithm"] = "tiled"
dac_notemp = pd.read_csv("./data/DAC_notemp.csv")
dac_notemp["algorithm"] = "dac_notemp"
dac_temp = pd.read_csv("./data/DAC_temp.csv")
dac_temp["algorithm"] = "dac_temp"
strassens = pd.read_csv("./data/strassens.csv")
strassens["algorithm"] = "strassens"

In [ ]:
vanilla_median = vanilla.groupby(["threads", "N", "algorithm"]).median().reset_index()
parfor_median = parfor.groupby(["threads", "N", "algorithm"]).median().reset_index()
tiled_median = tiled.groupby(["threads", "N", "param", "algorithm"]).median().reset_index()
dac_notemp_median = dac_notemp.groupby(["threads", "N", "param", "algorithm"]).median().reset_index()
dac_temp_median = dac_temp.groupby(["threads", "N", "param", "algorithm"]).median().reset_index()
strassens_median = strassens.groupby(["threads", "N", "param", "algorithm"]).median().reset_index()

dfs = [vanilla_median, parfor_median, tiled_median, dac_notemp_median, dac_temp_median, strassens_median]

In [ ]:
for df in dfs:
    # df["serial_runtime(ms)"] = df.apply(lambda r:
    #                                     df.query(f"N == {r['N']} and param == {r['param']} and threads == 1")
    #                                     ["runtime(ms)"].iloc[0], axis=1)
    df["serial_runtime(ms)"] = df.apply(lambda r: 
                                        vanilla_median.query(f"N == {r['N']}")["runtime(ms)"].iloc[0], axis=1)
    df["Speedup"] = df["serial_runtime(ms)"] / df["runtime(ms)"]
    df["Efficiency"] = df["Speedup"] / df["threads"]

In [ ]:
parfor_median["N"].unique(), parfor_median["threads"].unique()

In [ ]:
max_time = max(map(lambda df: df["runtime(ms)"].max(), dfs))
max_time

In [ ]:
all_runs = pd.concat(dfs).reset_index(drop=True)
all_runs.loc[all_runs.groupby(["threads", "N"])["runtime(ms)"].idxmin()]

In [ ]:
for thr, g1 in all_runs.query("algorithm != 'vanilla'").groupby("threads"):
    colors = sns.color_palette("pastel", 6)
    sns.lineplot(vanilla_median, x="N", y="runtime(ms)", label="vanilla", 
                c=colors[0], linewidth=3, marker="X", markersize=10)
    for i, (alg, g2) in enumerate(g1.groupby("algorithm")):
        sns.lineplot(g2, x="N", y="runtime(ms)", label=alg,
                     c=colors[i+1], errorbar=lambda x: (x.min(), x.max()),
                     linewidth=3, marker="X", markersize=10)
    t = f"Runtimes per Algorithm with threads = {thr}"
    plt.title(t, weight="bold")
    plt.xlim(64, g1["N"].max())
    plt.ylim(0, max_time)
    plt.gcf().set_size_inches(8, 6)
    plt.savefig("./figures/" + t + ".png", transparent=True)
    plt.show()

In [ ]:
def plot_runtimes(df, x, y, title):
    colors = sns.color_palette("Spectral", 6)
    t = f"Runtimes of the {title} Algorithm"
    sns.lineplot(df[df["threads"] == 1], x=x, y=y, label="1 thread", c=colors[0])
    for i, threads in enumerate([2, 4, 8, 12, 16]):
        sns.lineplot(df[df["threads"] == threads], x=x, y=y, label=f"{threads} threads", c=colors[i+1],
                     errorbar=lambda x: (x.min(), x.max()), marker="X", markersize=10)
    plt.title(t, weight="bold")
    plt.xlim(64, df[x].max())
    plt.ylim(0, max_time)
    plt.gcf().set_size_inches(8, 6)
    plt.savefig("./figures/" + t + ".png", transparent=True)
    plt.show()


def plot_strong_scaling(df, x, y, title, param_name=None):
    t = f"Strong Scaling of the {title} Algorithm"
    colors = sns.color_palette("pastel", len(df["param"].unique()))
    for p, c in zip(df["param"].unique(), colors):
        sns.lineplot(df[df["param"] == p], x=x, y=y, errorbar=lambda x: (x.min(), x.max()), linewidth=3,
                     label=f"Real {y}" + (f" ({param_name} = {p})" if param_name else ""), 
                     c=c, marker="X", markersize=10)
    if y == "Speedup":
        plt.axline((1, 1), (df[x].max(), df[x].max()), label=f"Ideal {y}", c="black")
    else:
        plt.axline((0, 1), (df[x].max(), 1), label=f"Ideal {y}", c="black")
    plt.title(t, weight="bold")
    plt.xlim(2, df[x].max())
    if y == "Speedup":
        plt.ylim(1, df[x].max())
    else:
        plt.ylim(0, 2)
    plt.gcf().set_size_inches(8, 6)
    plt.savefig("./figures/" + t + f" - {y}.png", transparent=True)
    plt.legend()
    if y == "Speedup":
        plt.grid()
    plt.show()


def plot_weak_scaling(df, x, y, title, param_name=None):
    t = f"Weak Scaling of the {title} Algorithm"
    colors = sns.color_palette("pastel", len(df["param"].unique()))
    for p, c in zip(df["param"].unique(), colors):
        subset = df.query(f"N == 64 * threads and param == {p}")
        sns.lineplot(subset, x=x, y=y, errorbar=lambda x: (x.min(), x.max()), linewidth=3,
                     label=f"Real {y}" + (f" ({param_name} = {p})" if param_name else ""), 
                     c=c, marker="X", markersize=10)
    if y == "Speedup":
        plt.axline((1, 1), (subset[x].max(), subset[x].max()), label=f"Ideal {y}", c="black")
    else:
        plt.axline((0, 1), (subset[x].max(), 1), label=f"Ideal {y}", c="black")
    plt.title(t, weight="bold")
    plt.xlim(2, subset[x].max())
    if y == "Speedup":
        plt.ylim(1, subset[x].max())
    else:
        plt.ylim(0, 2)
    plt.gcf().set_size_inches(8, 6)
    plt.savefig("./figures/" + t + f" - {y}.png", transparent=True)
    plt.legend()
    if y == "Speedup":
        plt.grid()
    plt.show()
    

In [ ]:
plot_runtimes(parfor_median, "N", "runtime(ms)", "Parallel For")
plot_runtimes(tiled_median, "N", "runtime(ms)", "Tiled")
plot_runtimes(dac_temp_median, "N", "runtime(ms)", "DAC (with temp matrix)")
plot_runtimes(dac_notemp_median, "N", "runtime(ms)", "DAC (without temp matrix)")
plot_runtimes(strassens_median, "N", "runtime(ms)", "Strassens")

In [ ]:
max_N = 512
plot_strong_scaling(parfor_median[parfor_median["N"] == max_N], "threads", "Speedup", "Parallel For")
plot_strong_scaling(tiled_median[tiled_median["N"] == max_N], "threads", "Speedup", "Tiled", "Tile Size")
plot_strong_scaling(dac_temp_median[dac_temp_median["N"] == max_N], "threads", "Speedup", "DAC (with temp matrix)", "Serial Cutoff")
plot_strong_scaling(dac_notemp_median[dac_notemp_median["N"] == max_N], "threads", "Speedup", "DAC (without temp matrix)", "Serial Cutoff")
plot_strong_scaling(strassens_median[strassens_median["N"] == max_N], "threads", "Speedup", "Strassens", "Serial Cutoff")

In [ ]:
plot_weak_scaling(parfor_median, "threads", "Speedup", "Parallel For")
plot_weak_scaling(tiled_median, "threads", "Speedup", "Tiled", "Tile Size")
plot_weak_scaling(dac_temp_median, "threads", "Speedup", "DAC (with temp matrix)", "Serial Cutoff")
plot_weak_scaling(dac_notemp_median, "threads", "Speedup", "DAC (without temp matrix)", "Serial Cutoff")
plot_weak_scaling(strassens_median, "threads", "Speedup", "Strassens", "Serial Cutoff")

In [ ]:
max_N = 512
plot_strong_scaling(parfor_median[parfor_median["N"] == max_N], "threads", "Efficiency", "Parallel For")
plot_strong_scaling(tiled_median[tiled_median["N"] == max_N], "threads", "Efficiency", "Tiled", "Tile Size")
plot_strong_scaling(dac_temp_median[dac_temp_median["N"] == max_N], "threads", "Efficiency", "DAC (with temp matrix)", "Serial Cutoff")
plot_strong_scaling(dac_notemp_median[dac_notemp_median["N"] == max_N], "threads", "Efficiency", "DAC (without temp matrix)", "Serial Cutoff")
plot_strong_scaling(strassens_median[strassens_median["N"] == max_N], "threads", "Efficiency", "Strassens", "Serial Cutoff")

In [ ]:
plot_weak_scaling(parfor_median, "threads", "Efficiency", "Parallel For")
plot_weak_scaling(tiled_median, "threads", "Efficiency", "Tiled", "Tile Size")
plot_weak_scaling(dac_temp_median, "threads", "Efficiency", "DAC (with temp matrix)", "Serial Cutoff")
plot_weak_scaling(dac_notemp_median, "threads", "Efficiency", "DAC (without temp matrix)", "Serial Cutoff")
plot_weak_scaling(strassens_median, "threads", "Efficiency", "Strassens", "Serial Cutoff")